This little program enables the autoimation of grade collection, checking for upcoming due dates and maybe something else if I get a request or an idea. 
Requirements:
Selenium: https://pypi.org/project/selenium/
Numpy: https://pypi.org/project/numpy/
Pandas: https://pypi.org/project/pandas/


This sets up the webdriver. This only works in firefox HOWEVER you can use chrome using this instead: 
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
**note that I havent tested it in chrome. not all web drivers are the same so your milage may vary. 

In [9]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(options=options)
options.add_argument("-headless")
options.add_argument('--no-sandbox')

This here sets up the base url of brightspace "https://schoolspecific.brightspace.com" is the typical url base. It also sets up the various buttons that need to be clicked though to get through MSFT login. the getPass import is a input that is passed to your password variable in order to keep it out of the source. 

In [10]:
import getpass # so you don't show your password in the sourcecode
email = 'w0499673@campus.nscc.ca'
base_url = 'https://nscconline.brightspace.com'
password = getpass.getpass()
email_field = (By.ID, 'i0116')
password_field = (By.ID, 'i0118')
next_button = (By.ID, 'idSIButton9')

This just runs through the MSFT login process inorder to get to the brightspace homepage

In [11]:
driver.get(base_url)
WebDriverWait(driver,10).until(EC.element_to_be_clickable(email_field)).send_keys(email)
WebDriverWait(driver,10).until(EC.element_to_be_clickable(next_button)).click()
WebDriverWait(driver,10).until(EC.element_to_be_clickable(password_field)).send_keys(password)
WebDriverWait(driver,10).until(EC.element_to_be_clickable(next_button)).click()
WebDriverWait(driver,10).until(EC.element_to_be_clickable(next_button)).click()

This block Iterates through a list of validated grade page urls. Nested loop then extracts the <table> contents from the HTML on each page. the second for loop removes the "Points" and the "comments and asscessments" column from each list dataframe.

In [12]:
grades_extensions = ["/d2l/lms/grades/my_grades/main.d2l?ou=299885",
                     "/d2l/lms/grades/my_grades/main.d2l?ou=297335",
                     "/d2l/lms/grades/my_grades/main.d2l?ou=295501",
                     "/d2l/lms/grades/my_grades/main.d2l?ou=295683",
                     "/d2l/lms/grades/my_grades/main.d2l?ou=297213"
                    ]
import pandas as pd
import io
import numpy as np
url = []
df = []
new_df=[]
#Iterates through list of grade page urls. Nested loop then extracts the <table> contents from the HTML on each page
for each in grades_extensions:
    url = base_url + each
    driver.get(url)
    
    for x in url:
        tbl = io.StringIO(driver.find_element(By.CSS_SELECTOR,".d2l-grid-wrapper").get_attribute('outerHTML'))
        df  = pd.read_html(tbl)
    #This clears out the columns "Points" and "Comments and Assessments". 
    for i in df:
        i.pop("Points")
        i.pop("Comments and Assessments")
        new_df.append(i)
df0 = new_df[0]
df1 = new_df[1]
df2 = new_df[2]
df3 = new_df[3]
df4 = new_df[4]

pd.concat([df0,df1,df2,df3,df4], axis=1).to_csv("test.csv", index=False)

driver.close()
